In [ ]:
%run "../Retropy_framework.ipynb"

In [ ]:
mdf = pd.read_csv("GemelNet.csv")
mdf["month_return"] = mdf["month_return"].astype(str)
mdf["month"] = pd.to_datetime(mdf["month"], format="%Y/%m/%d")
mdf["month_return"] = pd.to_numeric(mdf["month_return"].str.replace("%", ""), errors="coerce")

In [ ]:
#mdf = mdf.set_index("month")

def getValue(df):
    df = df.set_index("month")
    ser = df["month_return"].dropna().copy()
    if len(ser) == 0:
        return pd.Series()
    prevMonth = ser.index[0] + pd.DateOffset(months=-1)
    ser[prevMonth] = 0 # we want the serie sto start at 1.0
    ser = ser.sort_index()
    ser.index = ser.index + pd.DateOffset(months=1)
    return (1+ser.dropna()/100).cumprod()

#sym = mdf[mdf["ID"] == 1544]
#ym["month_return"]
#getValue(sym).plot()

def getKeren(id):
    df = mdf[mdf["ID"] == id]
    name = str(id) + " | " + df.iloc[0]["name"]
    s = getValue(df)
    usdils = getForex("USD", "ILS")
    s = (s/usdils).dropna()
    if len(s) > 0:
        s = s / s[0]
    return wrap(s, name)
    

### השוואה של קרן השתלמות מחקה ממדי חול איי-בי-איי למדדי חול

In [5]:
s = getKeren(1544)
bench = get("URTH")
bench = get("SPY:50|VGK:30|EEM:13|EWJ:7")
show(s, bench, VT, trim=True)

### השוואה של קרן השתלמות מחקה סנופי איי-בי-איי לסנופי

In [6]:
s = getKeren(2265)
bench = get("SPXNTR@IC")
x = (s / bench).dropna()
show(s, bench, trim=True)

Fetching SPXNTR@IC from IC .. DONE


### כל קרנות ההשתלמות מגמל.נט

In [8]:
allKranotIDs = set(mdf.loc[mdf["type"]=="קרן השתלמות", "ID"].values)
allKranot = []
for id in allKranotIDs:
    keren = getKeren(id)
    allKranot.append(keren)
    print(keren.name)
allKranot = [s for s in allKranot if len(s.s) > 0]

2048 |  *** מגדל השתלמות  הלכה
1537 | אינפיניטי השתלמות מניות בחו"ל
8704 | "אינטרגמל השתלמות"-מדד "תל בונד 20"
8705 | "אינטרגמל השתלמות"-מסלול מזומנים ושווי מזומנים
1542 | אקסלנס השתלמות בניהול אישי
2055 |  *** מיטב דש השתלמות חו"ל
1544 |  *** אי.בי.אי השתלמות פאסיבי - מדדי מניות חו"ל
1034 |  *** הראל השתלמות מסלול חו"ל
2058 | הלמן-אלדובי השתלמות שריעה
1038 | ילין לפידות קרן השתלמות מסלול אג"ח עד 25% מניות
7703 | אנליסט השתלמות כללי פאסיבי
2041 | ** מורים תיכוניים - מסלול הלכה
1059 |  *** פסגות שיא השתלמות פאסיבי-כללי
554 |  *** מיטב דש השתלמות עד 6 שנות וותק
2091 |  *** אקסלנס השתלמות אג"ח עד 20% מניות
558 |  *** אלטשולר שחם השתלמות אג"ח עד 15% במניות
2100 | אי.בי.אי השתלמות בניהול אישי
566 |  *** הראל השתלמות לטווח ארוך
(מסלול לאוכלוסיית יעד) 
8761 | הראל השתלמות IRA IRA
1083 | אינפיניטי השתלמות כללי
1084 | אינפיניטי השתלמות אג"ח
1085 | אינפיניטי השתלמות מניות בישראל
2114 | אינפיניטי השתלמות IRA בניהול אישי
579 |  *** מגדל השתלמות כללי
580 | ** שיבולת - השתלמות
1093 |  *** אלטשולר שח

481 | מיטב דש השתלמות  כללי פאסיבי
482 | ** מורים על יסודי
2018 |  *** פסגות שיא השתלמות  הלכה
484 | ** מורים וגננות - מסלול כללי
485 | ** מורים תיכוניים - מסלול כללי
8679 | אנליסט השתלמות אג"ח עד 20% במניות
2024 | הפניקס השתלמות בניהול אישי
1515 | פסגות השתלמות בניהול אישי
1523 |  *** הראל השתלמות מסלול כהלכה
2036 | ** מורים וגננות - מסלול אג"ח
2035 | ** מורים וגננות - מסלול הלכה
2038 | ** מורים תיכוניים - מסלול אג"ח
2039 | ** מורים וגננות - מסלול הלכה
2040 | ** מורים וגננות - מסלול אג"ח
2037 | ** מורים תיכוניים - מסלול הלכה
2042 | ** מורים תיכוניים - מסלול אג"ח
8701 | "אינטרגמל השתלמות"-מדד "מניות תל אביב 35"
1534 | כלל השתלמות הלכה
1535 | אנליסט השתלמות שקלי טווח קצר


### השוואה של כל קרנות ההשתלמות במסלול כללי לתיק 60\40 פשוט

In [12]:
lst = [s for s in allKranot if s.index[0].year < 2010 and "כללי" in s.name]

base = get("VT:60|VFITX:40")
lst = [(s).dropna() for s in lst]

show(*lst, base, ta=True, legend=False)

### השוואה של כל קרנות ההשתלמות במסלול כללי לתיק 60\40 פשוט - השוואה יחסית בין הקרנות

In [13]:
def getMedianSer(lst):
    df = pd.DataFrame([s.s for s in lst]).T
    return wrap(df.median(axis=1), "median")

lst = [s / s[0] for s in lst]
finals = [(s, s[-1]) for s in lst]
finals = sorted(finals, key=lambda x: x[1])
medianSer = finals[len(finals) // 2][0]
medianSer = getMedianSer(lst)
lst2 = [s / medianSer for s in lst]

show(*lst2, base, ta=True, legend=False)

In [14]:
# be sure to save before you publish
# when done, you will get a public link to an HTML file with the results
publish()

<IPython.core.display.Javascript object>

Publishing GemelNetPlay.ipynb ..
Failed!
